# XBeach profile evolution

Once your model finished running, it is time to have a look at the model netCDF outputs. XBeach generates a single NC file with all data in it. 

First we need to load the IPython libraries:

In [ ]:
%pylab inline
%matplotlib inline
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from JSAnimation import IPython_display

# Loading XBeach output

The netcdf file is located in the same location as your input files, we use the IPython netCDF4 functionality to load the data file. Using the `variables.keys()`, we can see all the parameters outputed by XBeach

In [ ]:
nc_data = netCDF4.Dataset('./default/xboutput.nc')
print nc_data.variables.keys()

To query the type/definition of each of the `netcdf` keys we can do:

In [ ]:
nc_data.variables['H']

# Get visualisation parameters

In [ ]:
# Export the stored time values
times = nc_data.variables['globaltime'][:]

# Export the X-axis values of the profile
xprofile = nc_data.variables['globalx'][0,:]

# Export the bed elevation
bed_elev = nc_data.variables['zb'][:,0,:]

# Export the wave height
wave_height = nc_data.variables['H'][:,0,:]

# Export the water level
water_lvl = nc_data.variables['zs'][:,0,:]

## Check parameters shape

The shape of an array gives us the dimension of the dataset we are looking at:

In [ ]:
print 'shape time values: ',times.shape
print 'shape X-axis profile: ',xprofile.shape
print 'shape bed elevation: ',bed_elev.shape
print 'shape wave height: ',wave_height.shape
print 'shape water level: ',water_lvl.shape

# Plotting XBeach model at a given time

To plot the model result for a given time we can define a function that will handle the process automatically. To define a function in IPython we use the following keyword: `def`

In [ ]:
# FID is the frame ID: an integer defining the time step we want to plot
def xbeach_frame(FID):
    
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(10,5), dpi=80) 
    axes = plt.gca()
    
    # Set extend of computational domain
    axes.set_xlim([0,3000])
    axes.set_ylim([-20,20])

    # Define the base of the model
    base = np.zeros(len(xprofile[:]))
    base.fill(-20)
    
    # Define bed elevation for given frame ID
    bed = bed_elev[FID,:]
    ax1.plot(xprofile[:],bed,linewidth=2.0,color=[139./255.,131./255.,120./255.])
    ax1.fill_between(xprofile[:], base, bed, where=bed >= base, facecolor=[1.0,0.9,0.6], interpolate=True)

    # Define top water surface  
    sea = water_lvl[FID,:] + wave_height[FID,:]
    ax1.plot(xprofile[:],sea,linewidth=2.0,color=[0./255.,0./255.,128./255.])
    ax1.fill_between(xprofile[:], bed, sea, where= sea > bed, facecolor=[0.7,0.9,1.0], interpolate=True)

    # Axes definitions
    tlt='XBeach model time:'+str(int(times[FID]))+' s'
    plt.title(tlt, fontsize=12)
    ax1.set_ylabel('elevation [m]', fontsize=12)
    ax1.set_xlabel('lenght [m]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    
    # Set legend
    names=['zb','H+zs']
    ax1.legend(names,loc='lower right', fontsize=11)
    plt.show()

Now we can simply call the function above for different time steps:

In [ ]:
xbeach_frame(500)
#xbeach_frame(2000)

# Making an animation of the results

To visualise an animation of the water and profile evolution over the duration of the model we can use the following cell

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = plt.axes(xlim=(0, 3000), ylim=(-20, 20))
plt.title('XBeach model run', fontsize=12)
ax.set_ylabel('elevation [m]', fontsize=12)
ax.set_xlabel('lenght [m]', fontsize=12)
plt.setp(ax.get_xticklabels(), fontsize=10)
plt.setp(ax.get_yticklabels(), fontsize=10)
    
name = ['initial zb','zb','H+zs']

ax.plot(xprofile[:],bed_elev[0,:],'--', lw=1,color='k')

line1, = ax.plot([], [], lw=2,color=[139./255.,131./255.,120./255.] )
line2, = ax.plot([], [], lw=2,color=[0./255.,0./255.,128./255.])

# Initialization function: plot the background of each frame
def init():
    line1.set_data([], [])
    line2.set_data([], [])
    return line1,line2, 

base = -20.
top = np.zeros(len(xprofile[:]))
top.fill(20.)

# Animation function.  This is called sequentially
def animate(i):
    x = xprofile[:]
    f1 = bed_elev[i,:]
    sea = wave_height[i,:]+water_lvl[i,:]
    f2 = sea
    line1.set_data(x, f1)
    line2.set_data(x, f2)
    ax.fill_between(x, base, top, where= top > base, facecolor=[1.,1.,1.], interpolate=True)
    ax.fill_between(x, base, f1, where=f1 >= base, facecolor=[1.0,0.9,0.6], interpolate=True)
    ax.fill_between(x, f1, f2, where= f2 > f1, facecolor=[0.7,0.9,1.0], interpolate=True)
    ax.legend(name,loc='lower right', fontsize=9)
    return line1,line2 

# Call the animator.  blit=True means only re-draw the parts that have changed.
animation.FuncAnimation(fig, animate, init_func=init, frames=np.arange(0, 2000, 20), interval=200, blit=True)

# Analyze results

What we did so far is setting-up and running a simple model and visualizing the results. The visualization was limited to a plain representation of the model output. Often, it is necessary to obtain insight in the overall characteristics of the model results in wave propagation and erosion progression in terms of volumes or retreat distances. The following section describes a few simple tools to extract these characteristics from the model output.


# Beach profile

From the animation above we can see that some erosion/deposition is happening around the 2700 m mark. We will take a closer look at what is happening first

In [ ]:
# FID is the frame ID: an integer defining the time step we want to plot
def profile_frame(FID):
    
    names=[]
    
    # Initial sea elevation (from params.txt)
    zso = 5
    
    # Number of profile to plot (start at 0)
    nbframe = len(FID)-1
    
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(15,7), dpi=80) 
    axes = plt.gca()
    
    # Set extend of visualisation domain
    axes.set_xlim([2600,2850])
    axes.set_ylim([-2,16])

    # Define the base of the model
    base = np.zeros(len(xprofile[:]))
    base.fill(-20)
  
    # Define top water surface using initial condition 
    #sea = np.zeros(len(xprofile[:]))
    #sea.fill(zso)
    #ax1.plot(xprofile[:],sea,linewidth=2.0,color=[0./255.,0./255.,128./255.])
    #ax1.fill_between(xprofile[:], bed_elev[nbframe,:], sea, where= sea > bed_elev[nbframe,:], facecolor=[0.7,0.9,1.0], interpolate=True)
  
    # Define bed elevation for given frame ID
    for i in range(nbframe+1):
        bed = bed_elev[FID[i],:]
        if i == 0:
            ax1.plot(xprofile[:],bed,'--',linewidth=2.0,color='k')
            names.append('initial bed at '+str(int(times[FID[i]]))+'s')
        elif i<nbframe:
            ax1.plot(xprofile[:],bed,'--',linewidth=1.0,color='k')
            names.append('bed at time '+str(int(times[FID[i]]))+'s')
        else:
            ax1.plot(xprofile[:],bed,linewidth=2.0,color='k')
            names.append('final bed at '+str(int(times[FID[i]]))+'s')
            sea = water_lvl[FID[i],:]
            ax1.plot(xprofile[:],sea,linewidth=2.0,color=[0./255.,0./255.,128./255.])
            names.append('water level at '+str(int(times[FID[i]]))+'s')
            ax1.fill_between(xprofile[:], bed, sea, where= sea > bed, facecolor=[0.7,0.9,1.0], interpolate=True)
    
    ax1.fill_between(xprofile[:], base, bed, where=bed >= base, facecolor=[1.0,0.9,0.6], interpolate=True)

    # Axes definitions
    tlt='Zoom of profile evolution through time close to the shoreline'
    plt.title(tlt, fontsize=12)
    ax1.set_ylabel('elevation [m]', fontsize=12)
    ax1.set_xlabel('lenght [m]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    
    # Set legend
    ax1.legend(names,loc='lower right', fontsize=11)
    plt.show()

In [ ]:
profile_frame([0,600,1200,1999])

# Hydrodynamics

We will compute hydrodynamic parameters like RMS wave heights over a cross-section. The same is done for orbital velocities and mean velocities. Also we will plot the Wave energy evolution over the profile as well as the dissipation of this energy along the profile. 

In [ ]:
# Wave height rms
Hrms = np.mean(nc_data.variables['H'][:,0,:], axis=0)
# Water velocity
ue = np.mean(nc_data.variables['ue'][:,0,:], axis=0)
# Wave energy
E = np.mean(nc_data.variables['E'][:,0,:], axis=0)
# Roller energy
R = np.mean(nc_data.variables['R'][:,0,:], axis=0)
# Energy dissipation
D = np.mean(nc_data.variables['D'][:,0,:], axis=0)
# Roller energy dissipation
Drol = np.mean(nc_data.variables['DR'][:,0,:], axis=0)

We define the series of plots that we will do in the next cell:

In [ ]:
def plot_HRMS():
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(15,3), dpi=80) 
    axes = plt.gca()
    # Set extend of computational domain
    axes.set_xlim([0,3000])
    axes.set_ylim([-0.5,5])
    ax1.plot(xprofile[:],Hrms,linewidth=2.0)
    # Axes definitions
    ax1.set_ylabel('elevation [m]', fontsize=12)
    ax1.set_xlabel('lenght [m]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    plt.grid()
    # Set legend
    names=['Hrms']
    ax1.legend(names,loc='upper right', fontsize=11)
    plt.show()
    
############################
def plot_Velocity():
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(15,3), dpi=80) 
    axes = plt.gca()
    # Set extend of computational domain
    axes.set_xlim([0,3000])
    axes.set_ylim([-2,0.5])
    ax1.plot(xprofile[:],ue,linewidth=2.0)
    # Axes definitions
    ax1.set_ylabel('velocity [m/s]', fontsize=12)
    ax1.set_xlabel('lenght [m]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    plt.grid()
    # Set legend
    names=['water velocity']
    ax1.legend(names,loc='lower right', fontsize=11)
    plt.show()
    
############################
def plot_Energy():
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(15,3), dpi=80) 
    axes = plt.gca()
    # Set extend of computational domain
    axes.set_xlim([0,3000])
    axes.set_ylim([min(E)-100,max(E)+100])
    ax1.plot(xprofile[:],E,linewidth=2.0)
    ax1.plot(xprofile[:],R*10,linewidth=2.0)
    # Axes definitions
    ax1.set_ylabel('energy [Nm/m2]', fontsize=12)
    ax1.set_xlabel('lenght [m]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    plt.grid()
    # Set legend
    names=['Wave energy', 'Roller energy x 10']
    ax1.legend(names,loc='upper right', fontsize=11)
    plt.show()
    
############################
def plot_Dissipation():
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(15,3), dpi=80) 
    axes = plt.gca()
    # Set extend of computational domain
    axes.set_xlim([0,3000])
    axes.set_ylim([min(Drol)-100,max(Drol)+100])
    ax1.plot(xprofile[:],D,linewidth=2.0)
    ax1.plot(xprofile[:],Drol,linewidth=2.0)
    # Axes definitions
    ax1.set_ylabel('energy dissipation [W/m2]', fontsize=12)
    ax1.set_xlabel('lenght [m]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    plt.grid()
    # Set legend
    names=['Wave dissipation', 'Roller dissipation']
    ax1.legend(names,loc='upper left', fontsize=11)
    plt.show()

We will not plot the dataset by calling the previously defined functions:

In [ ]:
plot_HRMS()
plot_Velocity()
plot_Energy()
plot_Dissipation()

# Sediment transports

We first define the output xbeach parameters that will be used:

In [ ]:
# Depth-averaged suspended concentration 
SuspConc = np.mean(nc_data.variables['ccg'][:,0,0,:], axis=0)
# Depth-averaged suspended equilibrium concentration 
SEqConc = np.mean(nc_data.variables['ceqsg'][:,0,0,:], axis=0)
# Depth-averaged bed equilibrium concentration
BEqConc = np.mean(nc_data.variables['ceqbg'][:,0,0,:], axis=0)
# Suspended sediment transport
SuspTrans = np.mean(nc_data.variables['Susg'][:,0,0,:], axis=0)

Then we define the plotting function

In [ ]:
def plot_SedConc():
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(15,3), dpi=80) 
    axes = plt.gca()
    # Set extend of computational domain
    axes.set_xlim([2600,2850])
    axes.set_ylim([0.,0.03])
    ax1.plot(xprofile[:],SuspConc,linewidth=2.0)
    ax1.plot(xprofile[:],SEqConc,linewidth=2.0)
    ax1.plot(xprofile[:],BEqConc*10,linewidth=2.0)
    # Axes definitions
    ax1.set_ylabel('Concentration [m3/m3]', fontsize=12)
    ax1.set_xlabel('lenght [m]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    plt.grid()
    # Set legend
    names=['Depth-averaged suspended concentration','Depth-averaged suspended equilibrium concentration ','Depth-averaged bed equilibrium concentration x 10']
    ax1.legend(names,loc='upper left', fontsize=11)
    plt.show()
    
############################
def plot_Transport():
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(15,3), dpi=80) 
    axes = plt.gca()
    # Set extend of computational domain
    axes.set_xlim([2600,2850])
    axes.set_ylim([min(SuspTrans)-0.01,max(SuspTrans)+0.01])
    ax1.plot(xprofile[:],SuspTrans,linewidth=2.0)
    # Axes definitions
    ax1.set_ylabel('transport [m2/s]', fontsize=12)
    ax1.set_xlabel('lenght [m]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    plt.grid()
    # Set legend
    names=['Suspended sediment transport']
    ax1.legend(names,loc='upper left', fontsize=11)
    plt.show()

Now we plot the graphs:

In [ ]:
plot_SedConc()
plot_Transport()

# Morphology

We do the same for plotting morphological evolution

In [ ]:
# Compute bed change
dx = xprofile[1]-xprofile[0]
bed = nc_data.variables['zb'][:,0,:]
Nbtime = len(bed[:,0])
zdiff = bed[Nbtime-1,:] - bed[0,:]

# Compute cumulative erosion, deposition over time
cumDep = np.zeros(Nbtime)
cumEro = np.zeros(Nbtime)
for t in range(1,Nbtime):
    diff = bed[t,:] - bed[t-1,:]
    eroIDs = np.where(diff<0)[0]
    depIDs = np.where(diff>0)[0]
    totero = -sum(diff[eroIDs])*len(eroIDs)*dx + cumEro[t-1]
    totdepo = sum(diff[depIDs])*len(depIDs)*dx + cumDep[t-1]
    cumEro[t] = totero
    cumDep[t] = totdepo
    
# Compute position of shoreline through time
shorepos = np.zeros(Nbtime)
zo = 5
for t in range(0,Nbtime):
    up = np.where(bed[t,:]>=zo)[0]
    a = (bed[t,up[1]]-bed[t,up[1]-1])/(xprofile[up[1]]-xprofile[up[1]-1])
    b = bed[t,up[1]-1] - a * xprofile[up[1]-1]
    shorepos[t] = (zo - b)/a 

In [ ]:
def plot_morpho():
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(15,3), dpi=80) 
    axes = plt.gca()
    # Set extend of computational domain
    axes.set_xlim([2550,2850])
    axes.set_ylim([min(zdiff)-0.25,max(zdiff)+0.25])
    ax1.plot(xprofile[:],zdiff,'-k',linewidth=3.0)
    ax1.fill_between(xprofile[:], zdiff, 0, where= zdiff > 0, facecolor=[204./255.,153./255.,255./255.], interpolate=True)
    ax1.fill_between(xprofile[:], zdiff, 0, where= zdiff < 0, facecolor=[0.7,0.9,1.0], interpolate=True)
    # Axes definitions
    ax1.set_ylabel('elevation [m]', fontsize=12)
    ax1.set_xlabel('lenght [m]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    plt.grid()
    # Set legend
    names=['elevation change']
    ax1.legend(names,loc='lower left', fontsize=11)
    plt.show()
    
def plot_erodep():
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(15,3), dpi=80) 
    axes = plt.gca()
    # Set extend of computational domain
    axes.set_xlim([0,1999])
    axes.set_ylim([min(cumEro),max(cumEro)+100])
    ax1.plot(np.arange(0,2000),cumEro,'-k',linewidth=3.0,color=[0.7,0.9,1.0])
    ax1.plot(np.arange(0,2000),cumDep,'-k',linewidth=3.0,color=[204./255.,153./255.,255./255.])
    # Axes definitions
    ax1.set_ylabel('cumulative change [m3/m]', fontsize=12)
    ax1.set_xlabel('time [step]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    plt.grid()
    # Set legend
    names=['cumulative erosion','cumulative deposition']
    ax1.legend(names,loc='lower right', fontsize=11)
    plt.show()
    
def plot_shore():
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(15,3), dpi=80) 
    axes = plt.gca()
    # Set extend of computational domain
    axes.set_xlim([0,1999])
    axes.set_ylim([min(shorepos)-0.5,max(shorepos)+0.5])
    ax1.plot(np.arange(0,2000),shorepos,'-',linewidth=3.0,color=[1.0,0.9,0.6])
    # Axes definitions
    ax1.set_ylabel('position [m]', fontsize=12)
    ax1.set_xlabel('time [step]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    plt.grid()
    # Set legend
    names=['shoreline position']
    ax1.legend(names,loc='upper left', fontsize=11)
    plt.show()

In [ ]:
plot_morpho()
plot_erodep()
plot_shore()